In [2]:
import numpy as np
import math
import itertools
import picos as pic

In [3]:
def Swap_multiplication(A,B):
    
    C = tuple(set(A)^set(B))
        
    return C 


def K_fun(M,K,N):
    
    Fun=0
    
    for q in range(0,M+1):
        Fun+= math.comb(N-K,M-q)*math.comb(K,q)*(-1)**q
    return Fun 

D=2 ### Dimension ###

n=4 ### n of qudits ###
 
k=2 ### Partitions ###

d=2**k ### Total number of reductions ###

Generators=list(range(0, k))

Elements=[]

for i in range(n+1):
    Elements += (list(itertools.combinations(Generators, i)))

#print(Elements)

Elements_R={new_label:old_label for old_label, new_label in enumerate(Elements)}

rho=[]

length=0

for i in range(0,int(k/2)+1):

        rho+=[(1/D**(i))]*(math.comb(k, i))

        length+=math.comb(k,i)
        
Rep=[]

for i in range(d):
    for j in range(d):
        C=Swap_multiplication(Elements[i],Elements[j])
        Rep.append([C,i,j])  

Rep=sorted(Rep)


##### SDP #####


pic.available_solvers()

F = pic.SymmetricVariable('F', (d,d)) # the moment matrix

M=np.zeros((d, d))

sdp = pic.Problem()

sdp.set_objective('find') 



##### Adding AME purities #####

sdp.add_list_of_constraints([F[0,i] == rho[i] for i in range(length)])

#sdp.add_list_of_constraints([F[0,i] == rho[i] for i in range(d)])

for i in range(length): M[0,i]=rho[i] 


##### Adding K-L constrains #####

for i in range(0,length):

    S_c=Swap_multiplication(Elements[i],Elements[d-1])
    
    sdp.add_constraint(F[0,Elements_R[S_c]]==F[0,i])
    
    M[0][Elements_R[S_c]]=M[0][i]
    

##### Adding NPA constrains #####

sdp.add_constraint(F>>0)
           
    
for rep in Rep:
    if rep==Rep[0]:  
        
        rep_old=rep.copy()
        
    else:

        if set(rep[0])==set(rep_old[0]):
        
            sdp.add_constraint(F[rep[1],rep[2]]==F[rep_old[1],rep_old[2]])
            M[rep[1],rep[2]]=M[rep_old[1],rep_old[2]]
        else:
            rep_old=rep.copy() 

#print(sdp)


##### Adding Shadow constrains #####
    

for j in range(k+1):
    sdp.add_constraint(sum([K_fun(k-j,i,k)*F[Elements_R[comb],0]
                                for i in range(k+1) for comb in itertools.combinations(Generators, i)]) >> 0)
        
        
    #print(sum([K_fun(k-j,i,k)*F[Elements_R[comb],0]
    #           for i in range(k+1) for comb in itertools.combinations(Generators, i)]))

##### Adding PPT constrains #####
    
for m in range(k+1):
    sdp.add_constraint(sum([(-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*F[0,Elements_R[comb]] for j in range(k+1) 
                             for comb in itertools.combinations(Generators, j)])  >> 0)

#print([K_fun(0,i,k) for i in range(k+1) for comb in itertools.combinations(Generators, i)])
#print(sdp)


#### Check matrix ####
                   
print(M) 
#print(sdp)                   
    
sol = sdp.solve(solver='mosek', verbosity=True)
      
                            
np.round(np.array(F),2)
                            

[[1.  0.5 0.5 1. ]
 [0.5 1.  1.  0.5]
 [0.5 1.  1.  0.5]
 [1.  0.5 0.5 1. ]]
           PICOS 2.4.11            
Problem type: Feasibility Problem.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. MOSEKSolver
Applying ExtraOptions.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 24              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 7               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear 

SolutionFailure: Code 3: Primal solution state claimed infeasible but optimality is required (primals=True).